In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/onlineretail/OnlineRetail.csv


# 1. Introduction

## **About the Data**

This dataset contains all purchases made for an online retail company based in the UK during an eight month period.

## **Goals**

1. **Customer Segmentation**:

	* Utilize RFM (Recency, Frequency, Monetary) analysis to segment customers based on their purchasing behavior.
	* Identify different customer groups such as loyal customers, high spenders, and potential churners.
    
2.	**Understanding Customer Behavior**:

	* Gain insights into customer purchasing patterns and trends.
	* Determine the recency of customer purchases, the frequency of their transactions, and their overall monetary contribution.
    
3.	**Targeted Marketing Strategies**:

	* Develop tailored marketing strategies for different customer segments.
	* Enhance customer engagement and retention by delivering personalized offers and promotions.
    
4.	**Revenue Maximization**:

	* Identify high-value customers who contribute significantly to revenue.
	* Develop strategies to nurture these customers and increase their lifetime value.
    
5.	**Customer Retention**:

	* Detect potential churners and create retention strategies to prevent customer attrition.
	* Understand the factors leading to customer churn and address them proactively.
    
6.	**Business Decision Making**:

	* Provide actionable insights to the business for making informed decisions.
	* Use RFM analysis to guide product recommendations, inventory management, and promotional activities.

By achieving these goals, the project aims to **enhance the overall customer experience, improve customer loyalty, and drive business growth through data-driven decision making.**

## **Attribute Information**

- **InvoiceNo: (Identifier)**

A unique identifier assigned to each transaction. Each row in the dataset corresponds to a single transaction.

- **StockCode: (Identifier)**

A unique identifier assigned to each distinct product.

- **Description: (Variable)**

A textual description of the product.

- **Quantity: (Variable)**

The quantity of each product per transaction. This represents the number of units sold per transaction.

- **InvoiceDate: (Variable)**

The date and time when a transaction was generated.

- **UnitPrice: (Variable)**

The unit price of the product. This represents the price per unit in GBP (British pounds).

- **Revenue: (Calculated Variable)**

The Revenue of each transaction. This represents the Quantity x Unit Price in GBP (British pounds).

- **CustomerID: (Identifier)**

A unique identifier assigned to each customer. Each customer can have multiple transactions associated with their ID.

- **Country: (Variable)**

The country where the customer resides.

## Additional Variables for RFM Analysis:

- **Recency: (Calculated Variable)**

The number of days since the customer's last purchase.

- **Frequency: (Calculated Variable)**

The total number of transactions made by the customer.

- **Monetary: (Calculated Variable)**

The total monetary value of the customer's purchases.

# 2. Importing Libraries

In [2]:
# basic
import pandas as pd
import numpy as np

# viz
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
from matplotlib.colors import LinearSegmentedColormap
from mpl_toolkits.mplot3d import Axes3D
import seaborn as sns
import plotly.express as px



# 3. Data Understanding

## 3.1 Load the Data

In [3]:
df = pd.read_csv("/kaggle/input/onlineretail/OnlineRetail.csv",encoding='unicode_escape')
df.head(25)

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/2010 8:26,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
5,536365,22752,SET 7 BABUSHKA NESTING BOXES,2,12/1/2010 8:26,7.65,17850.0,United Kingdom
6,536365,21730,GLASS STAR FROSTED T-LIGHT HOLDER,6,12/1/2010 8:26,4.25,17850.0,United Kingdom
7,536366,22633,HAND WARMER UNION JACK,6,12/1/2010 8:28,1.85,17850.0,United Kingdom
8,536366,22632,HAND WARMER RED POLKA DOT,6,12/1/2010 8:28,1.85,17850.0,United Kingdom
9,536367,84879,ASSORTED COLOUR BIRD ORNAMENT,32,12/1/2010 8:34,1.69,13047.0,United Kingdom


In [4]:
df.shape

(541909, 8)

In [5]:
print("\nData Types and Missing Values:")
print(df.info())


Data Types and Missing Values:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541909 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   InvoiceNo    541909 non-null  object 
 1   StockCode    541909 non-null  object 
 2   Description  540455 non-null  object 
 3   Quantity     541909 non-null  int64  
 4   InvoiceDate  541909 non-null  object 
 5   UnitPrice    541909 non-null  float64
 6   CustomerID   406829 non-null  float64
 7   Country      541909 non-null  object 
dtypes: float64(2), int64(1), object(5)
memory usage: 33.1+ MB
None


In [6]:
# set to datetime
df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'])

## 3.2 Null Values Checking

In [7]:
data_null_total = pd.DataFrame(df.isna().sum()).T.rename({0:'total null'})
data_null_total

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
total null,0,0,1454,0,0,0,135080,0


## 3.3 Null Values Handling 

- **Drop Rows with Null CustomerID**:

CustomerID is a critical piece of information for customer segmentation. Rows with null values in this column cannot be attributed to any customer, so it’s best to drop these rows.

- **Handle Null Description**: 

The Description column might be useful for certain types of analysis, but it’s not critical for RFM analysis. I prefer to ignore it at the moment.

In [8]:
# Drop rows where CustomerID is null
df_cleaned = df.dropna(subset=['CustomerID'])

# Display the number of remaining rows
print(f"Remaining rows after cleaning: {df_cleaned.shape[0]}")

Remaining rows after cleaning: 406829


In [9]:
df_cleaned['CustomerID'].astype(str)

0         17850.0
1         17850.0
2         17850.0
3         17850.0
4         17850.0
           ...   
541904    12680.0
541905    12680.0
541906    12680.0
541907    12680.0
541908    12680.0
Name: CustomerID, Length: 406829, dtype: object

## 3.4 Negative Values Checking

In [10]:
print("Summary Statistics:")
print(df_cleaned.describe())

Summary Statistics:
            Quantity                    InvoiceDate      UnitPrice  \
count  406829.000000                         406829  406829.000000   
mean       12.061303  2011-07-10 16:30:57.879207424       3.460471   
min    -80995.000000            2010-12-01 08:26:00       0.000000   
25%         2.000000            2011-04-06 15:02:00       1.250000   
50%         5.000000            2011-07-31 11:48:00       1.950000   
75%        12.000000            2011-10-20 13:06:00       3.750000   
max     80995.000000            2011-12-09 12:50:00   38970.000000   
std       248.693370                            NaN      69.315162   

          CustomerID  
count  406829.000000  
mean    15287.690570  
min     12346.000000  
25%     13953.000000  
50%     15152.000000  
75%     16791.000000  
max     18287.000000  
std      1713.600303  


## 3.5 Negative Values Handling

In [11]:
df_cleaned = df_cleaned[df_cleaned['UnitPrice']>0]
df_cleaned = df_cleaned[df_cleaned['Quantity']>0]

In [12]:
print(f"Remaining rows after cleaning: {df_cleaned.shape[0]}")

Remaining rows after cleaning: 397884


In [13]:
print("Summary Statistics:")
print(df_cleaned.describe())

Summary Statistics:
            Quantity                    InvoiceDate      UnitPrice  \
count  397884.000000                         397884  397884.000000   
mean       12.988238  2011-07-10 23:41:23.511023360       3.116488   
min         1.000000            2010-12-01 08:26:00       0.001000   
25%         2.000000            2011-04-07 11:12:00       1.250000   
50%         6.000000            2011-07-31 14:39:00       1.950000   
75%        12.000000            2011-10-20 14:33:00       3.750000   
max     80995.000000            2011-12-09 12:50:00    8142.750000   
std       179.331775                            NaN      22.097877   

          CustomerID  
count  397884.000000  
mean    15294.423453  
min     12346.000000  
25%     13969.000000  
50%     15159.000000  
75%     16795.000000  
max     18287.000000  
std      1713.141560  


# 4. Exploratory Data Analysis

In [14]:
df_cleaned['Revenue'] = df_cleaned['Quantity'] * df_cleaned['UnitPrice']
df_cleaned.head(25)

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,Revenue
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom,15.30
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom,22.00
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34
5,536365,22752,SET 7 BABUSHKA NESTING BOXES,2,2010-12-01 08:26:00,7.65,17850.0,United Kingdom,15.30
6,536365,21730,GLASS STAR FROSTED T-LIGHT HOLDER,6,2010-12-01 08:26:00,4.25,17850.0,United Kingdom,25.50
7,536366,22633,HAND WARMER UNION JACK,6,2010-12-01 08:28:00,1.85,17850.0,United Kingdom,11.10
8,536366,22632,HAND WARMER RED POLKA DOT,6,2010-12-01 08:28:00,1.85,17850.0,United Kingdom,11.10
9,536367,84879,ASSORTED COLOUR BIRD ORNAMENT,32,2010-12-01 08:34:00,1.69,13047.0,United Kingdom,54.08


## 4.1 Top Countries by Trx

In [15]:
# Calculate the transaction count per country
country_counts = df_cleaned['Country'].value_counts().head(10).reset_index()
country_counts.columns = ['Country', 'TransactionCount']

# Create a horizontal bar chart
fig = px.bar(
    country_counts,
    x='TransactionCount',
    y='Country',
    orientation='h',
    title='Top 10 Countries by Transaction Count',
#     color='Country',
    color_discrete_sequence=['#87bc45'] 
)

# Update layout for better visualization
fig.update_layout(
    xaxis_title='Transaction Count',
    yaxis_title='Country',
    yaxis=dict(categoryorder='total ascending'),
    showlegend=False
)

fig.show()

>The highest transaction generated by UK, home of the Company it self.

## 4.2 Top 10 Countries by Revenue

In [16]:
# Calculate the total revenue per country
country_revenue = df_cleaned.groupby('Country')['Revenue'].sum().reset_index()

# Get the top 10 countries by revenue
top_countries_revenue = country_revenue.nlargest(10, 'Revenue')

# Create a horizontal bar chart
fig = px.bar(
    top_countries_revenue,
    x='Revenue',
    y='Country',
    orientation='h',
    title='Top 10 Countries by Revenue',
#     color='Country',
    color_discrete_sequence=['#ef9b20'] 
)

# Update layout for better visualization
fig.update_layout(
    xaxis_title='Revenue (GBP)',
    yaxis_title='Country',
    yaxis=dict(categoryorder='total ascending'),
    showlegend=False
)

fig.show()

>Here can be seen: Netherlands although having less trx counts, it generates more Revenue to the Company more than Germany and France

## 4.3 Month to Month Trx

In [17]:
# Extract year and month from InvoiceDate
df_cleaned['YearMonth'] = df_cleaned['InvoiceDate'].dt.to_period('M')

# Group by YearMonth and count the number of transactions
monthly_transactions = df_cleaned.groupby('YearMonth')['InvoiceNo'].nunique().reset_index()

# Convert YearMonth to datetime for better plotting
monthly_transactions['YearMonth'] = monthly_transactions['YearMonth'].dt.to_timestamp()

# Create a bar chart
fig = px.bar(
    monthly_transactions,
    x='YearMonth',
    y='InvoiceNo',
    title='Month-to-Month Transactions',
    labels={'YearMonth': 'Month', 'InvoiceNo': 'Number of Transactions'},
    text='InvoiceNo'
)

# Standardize decimals and display all months on x-axis
fig.update_traces(marker_color='#87bc45', textposition='outside', texttemplate='%{text}')
fig.update_layout(
    xaxis_title='Month',
    yaxis_title='Number of Transactions',
    showlegend=False,
    bargap=0.2
#     ,xaxis=dict(tickmode='linear')
)

fig.show()

- There is a general increase in transactions leading up to the peak in November 2011, suggesting a possible seasonal trend or a successful promotional campaign during that period.

- After the peak, there is a significant drop in December 2011.

## 4.4 Month to Month Revenue

In [18]:
# Extract year and month from InvoiceDate
df_cleaned['YearMonth'] = df_cleaned['InvoiceDate'].dt.to_period('M')

# Group by YearMonth and sum the revenue
monthly_revenue = df_cleaned.groupby('YearMonth')['Revenue'].sum().reset_index()

# Convert YearMonth to datetime for better plotting
monthly_revenue['YearMonth'] = monthly_revenue['YearMonth'].dt.to_timestamp()

# Create a bar chart
fig = px.bar(
    monthly_revenue,
    x='YearMonth',
    y='Revenue',
    title='Month-to-Month Revenue',
    labels={'YearMonth': 'Month', 'Revenue': 'Revenue'},
    text='Revenue'
)

# Update layout for better visualization
fig.update_traces(marker_color='#ef9b20', textposition='outside', texttemplate='%{text:.2f}')
fig.update_layout(
    xaxis_title='Month',
    yaxis_title='Revenue (GBP)',
    showlegend=False,
    bargap=0.2
)

fig.show()

## 4.5 Month to Month Revenue x Trx

In [19]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Extract year and month from InvoiceDate
df_cleaned['YearMonth'] = df_cleaned['InvoiceDate'].dt.to_period('M')

# Group by YearMonth and sum the revenue
monthly_revenue = df_cleaned.groupby('YearMonth')['Revenue'].sum().reset_index()

# Group by YearMonth and count the number of transactions
monthly_transactions = df_cleaned.groupby('YearMonth')['InvoiceNo'].nunique().reset_index()

# Convert YearMonth to datetime for better plotting
monthly_revenue['YearMonth'] = monthly_revenue['YearMonth'].dt.to_timestamp()
monthly_transactions['YearMonth'] = monthly_transactions['YearMonth'].dt.to_timestamp()

# Create a dual-axis plot
fig = make_subplots(specs=[[{"secondary_y": True}]])

# Add trx bar chart to the plot
fig.add_trace(
    go.Bar(
        x=monthly_transactions['YearMonth'],
        y=monthly_transactions['InvoiceNo'],
        name='Transactions',
        text=monthly_transactions['InvoiceNo'],
        textposition='outside',
        marker_color='#87bc45'
    ),
    secondary_y=False,
)

# Add revenue line chart to the plot
fig.add_trace(
    go.Scatter(
        x=monthly_revenue['YearMonth'],
        y=monthly_revenue['Revenue'],
        name='Revenue',
        mode='lines+markers',
        marker=dict(color='#ef9b20'),
        texttemplate='%{text:.2f}'
    ),
    secondary_y=True,
)

# Update layout for better visualization
fig.update_layout(
    title_text='Month-to-Month Revenue and Transactions',
    xaxis_title='Month',
    bargap=0.2,
)

# Set y-axis titles
fig.update_yaxes(title_text="Revenue (GBP)", secondary_y=True)
fig.update_yaxes(title_text="Number of Transactions", secondary_y=False)

# Show the figure
fig.show()

> Revenue and Trx went along during month to month observation

## 4.6 Weekdays/Weekend Comparison by Trx Counts and Revenue

In [20]:
# Create a new column for Day of the Week
df_cleaned['DayOfWeek'] = df_cleaned['InvoiceDate'].dt.dayofweek

# Categorize as Weekday or Weekend
df_cleaned['WeekdayOrWeekend'] = df_cleaned['DayOfWeek'].apply(lambda x: 'Weekend' if x >= 5 else 'Weekday')

# Group by WeekdayOrWeekend and sum the revenue
revenue_by_day_type = df_cleaned.groupby('WeekdayOrWeekend')['Revenue'].sum().reset_index()

# Group by WeekdayOrWeekend and count the number of transactions
trx_counts_by_day_type = df_cleaned.groupby('WeekdayOrWeekend')['InvoiceNo'].nunique().reset_index()

# Create subplots for dual-axis chart
fig = make_subplots(specs=[[{"secondary_y": True}]])


# Add bar chart for transaction counts
fig.add_trace(
    go.Bar(
        x=trx_counts_by_day_type['WeekdayOrWeekend'],
        y=trx_counts_by_day_type['InvoiceNo'],
        name='Transaction Counts',
        marker_color='#87bc45',
        text=trx_counts_by_day_type['InvoiceNo'],
        textposition='outside',
        texttemplate='%{text:.0f}'
    ),
    secondary_y=False,
)

# Add line chart for revenue
fig.add_trace(
    go.Scatter(
        x=revenue_by_day_type['WeekdayOrWeekend'],
        y=revenue_by_day_type['Revenue'],
        name='Revenue',
        marker_color='#ef9b20',
        mode='lines+markers',
        text=revenue_by_day_type['Revenue'],
        textposition='top center',
        texttemplate='%{text:.2f}'
    ),
    secondary_y=True,
)

# Update layout for better visualization
fig.update_layout(
    title='Transaction Counts and Revenue: Weekday vs Weekend',
    xaxis_title='Type of Day',
    yaxis_title='Number of Transactions',
    yaxis2_title='Revenue (GBP)',
    bargap=0.2,
    showlegend=True,
)

# Show plot
fig.show()

- Higher Activity on Weekdays:

The business experiences significantly higher transaction counts and revenue on weekdays. This suggests that customers are more active in making purchases during the workweek.

- Weekend Strategy:

The lower activity on weekends could indicate an opportunity to boost sales through targeted marketing campaigns, promotions, or special offers.

## 4.7 Order Placement by Hour of The Day

In [21]:
# Extract the hour from InvoiceDate
df_cleaned['Hour'] = df_cleaned['InvoiceDate'].dt.hour

# Group by Hour and count the number of orders
orders_by_hour = df_cleaned.groupby('Hour')['InvoiceNo'].nunique().reset_index()

# Create a bar chart
fig = px.bar(
    orders_by_hour,
    x='Hour',
    y='InvoiceNo',
    title='Orders Placement by Hour of the Day',
    labels={'Hour': 'Hour of the Day', 'InvoiceNo': 'Number of Orders'},
    text='InvoiceNo',
    color='InvoiceNo',  # Use 'InvoiceNo' for color scale
    color_continuous_scale='Blues'  # Use gradient blue color scale
)

# Update layout for better visualization
fig.update_traces(textposition='outside', texttemplate='%{text:.0f}')
fig.update_layout(
    xaxis=dict(tickmode='linear', tick0=0, dtick=1),  # Show all hours on the x-axis
    xaxis_title='Hour of the Day',
    yaxis_title='Number of Orders',
    showlegend=False,
    bargap=0.2
)

fig.show()

# 5. RFM Analysis

## 5.1 Calculate RFM Metrics

In [22]:
import datetime as dt

# Set snapshot date to the day after the last transaction
snapshot_date = df_cleaned['InvoiceDate'].max() + dt.timedelta(days=1)

# Calculate Recency, Frequency, and Monetary value for each customer
rfm = df_cleaned.groupby('CustomerID').agg({
    'InvoiceDate': lambda x: (snapshot_date - x.max()).days,
    'InvoiceNo': 'nunique',
    'Revenue': 'sum'
}).reset_index()

# Rename columns
rfm.columns = ['CustomerID', 'Recency', 'Frequency', 'Monetary']

# Display the first few rows
rfm.head(15)

,CustomerID,Recency,Frequency,Monetary
0,12346.0,326,1,77183.60
1,12347.0,2,7,4310.00
2,12348.0,75,4,1797.24
3,12349.0,19,1,1757.55
4,12350.0,310,1,334.40
5,12352.0,36,8,2506.04
6,12353.0,204,1,89.00
7,12354.0,232,1,1079.40
8,12355.0,214,1,459.40
9,12356.0,23,3,2811.43


## 5.2 RFM Scores

In [23]:
# Perform RFM segmentation
def rfm_segmentation(data):
    # Segment Recency
    data['R'] = pd.qcut(data['Recency'], 5, labels=[5, 4, 3, 2, 1], duplicates='drop')
    
    # Segment Frequency
    data['F'] = pd.qcut(data['Frequency'].rank(method='first'), 5, labels=[1, 2, 3, 4, 5], duplicates='drop')
    
    # Segment Monetary
    data['M'] = pd.qcut(data['Monetary'], 5, labels=[1, 2, 3, 4, 5], duplicates='drop')
    
    # Calculate RFM Score
    data['RFM_Score'] = data['R'].astype(str) + data['F'].astype(str) + data['M'].astype(str)
    
    return data

# Segment customers
rfm = rfm_segmentation(rfm)

# Display the first few rows
rfm.head(15)

,CustomerID,Recency,Frequency,Monetary,R,F,M,RFM_Score
0,12346.0,326,1,77183.60,1,1,5,115
1,12347.0,2,7,4310.00,5,5,5,555
2,12348.0,75,4,1797.24,2,4,4,244
3,12349.0,19,1,1757.55,4,1,4,414
4,12350.0,310,1,334.40,1,1,2,112
5,12352.0,36,8,2506.04,3,5,5,355
6,12353.0,204,1,89.00,1,1,1,111
7,12354.0,232,1,1079.40,1,1,4,114
8,12355.0,214,1,459.40,1,1,2,112
9,12356.0,23,3,2811.43,4,3,5,435


## 5.3 Segment the Customers

In [24]:
# Define RFM score segments
def rfm_segment(data):
    segment = []
    for score in data['RFM_Score']:
        if score == '555':
            segment.append('Champions')
        elif score[0] == '5' or score[1] == '5' or score[2] == '5':
            segment.append('Loyal Customers')
        elif score[0] == '4' or score[1] == '4' or score[2] == '4':
            segment.append('Potential Loyalist')
        elif score[0] == '3' or score[1] == '3' or score[2] == '3':
            segment.append('New Customers')
        elif score[0] == '2' or score[1] == '2' or score[2] == '2':
            segment.append('Need Attention')
        else:
            segment.append('At Risk')
    data['Segment'] = segment
    return data

# Apply the segment function
rfm = rfm_segment(rfm)

# Display the first few rows with segments
rfm.head(15)

,CustomerID,Recency,Frequency,Monetary,R,F,M,RFM_Score,Segment
0,12346.0,326,1,77183.60,1,1,5,115,Loyal Customers
1,12347.0,2,7,4310.00,5,5,5,555,Champions
2,12348.0,75,4,1797.24,2,4,4,244,Potential Loyalist
3,12349.0,19,1,1757.55,4,1,4,414,Potential Loyalist
4,12350.0,310,1,334.40,1,1,2,112,Need Attention
5,12352.0,36,8,2506.04,3,5,5,355,Loyal Customers
6,12353.0,204,1,89.00,1,1,1,111,At Risk
7,12354.0,232,1,1079.40,1,1,4,114,Potential Loyalist
8,12355.0,214,1,459.40,1,1,2,112,Need Attention
9,12356.0,23,3,2811.43,4,3,5,435,Loyal Customers


## 5.4 RFM Segment Visualization

In [25]:
color_map = {
    'Potential Loyalist': 'lightcoral',
    'Loyal Customers': 'lightblue',
    'New Customers': 'lightgreen',
    'Need Attention': 'lightyellow',
    'Champions': 'gold',
    'At Risk': 'lightgrey'
}

In [26]:
# Count the number of customers in each segment
segment_counts = rfm['Segment'].value_counts().reset_index()
segment_counts.columns = ['Segment', 'Count']

# Create a bar chart
fig = px.bar(
    segment_counts,
    x='Segment',
    y='Count',
    title='Customer Segment Counts',
    labels={'Segment': 'Customer Segment', 'Count': 'Number of Customers'},
    text='Count',
    color='Segment',
    color_discrete_map=color_map
)

# Update layout for better visualization
fig.update_traces(textposition='outside', texttemplate='%{text:.0f}')
fig.update_layout(
    xaxis_title='Customer Segment',
    yaxis_title='Number of Customers',
    showlegend=False,
    bargap=0.2
)

fig.show()

/opt/conda/lib/python3.10/site-packages/plotly/express/_core.py:2065: FutureWarning:

When grouping with a length-1 list-like, you will need to pass a length-1 tuple to get_group in a future version of pandas. Pass `(name,)` instead of `name` to silence this warning.



## 5.5 RFM Analysis Interpretation

### 1. Potential Loyalist (1147 customers):

- Characteristics: These customers are on the verge of becoming loyal customers. They have made recent purchases and have a moderate to high frequency of transactions.
- Strategy: Engage these customers with personalized marketing campaigns, loyalty programs, and incentives to convert them into loyal customers.

### 2. Loyal Customers (1130 customers):

- Characteristics: These customers have high frequency and high monetary value. They consistently make purchases and contribute significantly to the revenue.
- Strategy: Maintain and strengthen relationships with these customers by providing exclusive offers, early access to new products, and superior customer service to ensure their continued loyalty.

### 3. New Customers (889 customers):

- Characteristics: These are recent customers with a low to moderate number of purchases. They have recently engaged with the company and are in the early stages of the customer lifecycle.
- Strategy: Nurture these customers with welcome emails, onboarding programs, and targeted promotions to encourage repeat purchases and move them towards becoming loyal customers.

### 4. Need Attention (641 customers):

- Characteristics: These customers have a moderate frequency and monetary value but haven’t made recent purchases. They are at risk of churning if not re-engaged.
- Strategy: Re-engage these customers with win-back campaigns, personalized offers, and reminders about the benefits of your products or services to rekindle their interest.

### 5. Champions (348 customers):

- Characteristics: These customers are the best and most valuable. They have high recency, frequency, and monetary values. They are the top spenders and frequent buyers.
- Strategy: Reward these customers with VIP programs, special recognition, and exclusive benefits to maintain their high level of satisfaction and encourage them to become brand advocates.

### 6. At Risk (183 customers):

- Characteristics: These customers have a high monetary value but have not made recent purchases. They are at high risk of churning and need immediate attention.
- Strategy: Implement targeted retention strategies such as personalized re-engagement campaigns, special discounts, and feedback requests to understand and address their reasons for disengagement.

## 5.6 Insights for Business Decisions:

### 1. Focus on Potential Loyalists and Loyal Customers:

Given that these two segments have the highest counts, strategies to convert potential loyalists into loyal customers and retain loyal customers will significantly impact overall revenue.

### 2. Re-engagement Campaigns:

The "Need Attention" and "At Risk" segments need targeted re-engagement campaigns to reduce churn and recover lost revenue.

### 3. Nurturing New Customers:

Efforts to onboard and nurture new customers will help in increasing their lifetime value and moving them into higher-value segments over time.

### 4. Rewarding Champions:

Maintaining the satisfaction of champion customers through exclusive rewards and recognition will encourage repeat purchases and positive word-of-mouth marketing.